In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [21]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)
epochs = 10

def get_data(training_data, test_data):
    return (
        DataLoader(training_data), #, shuffle=True),
        DataLoader(test_data),
    )

train_dl, valid_dl = get_data(training_data, test_data)

Using cpu device


In [48]:
x1,y1 = next(iter(train_dl))
x1, y1 =x1.flatten(), y1.squeeze()
x1.shape, y1.shape

(torch.Size([784]), torch.Size([]))

In [3]:
state_dict = torch.load('model.pth')
state_dict.keys()

odict_keys(['linear_relu_stack.0.weight', 'linear_relu_stack.0.bias', 'linear_relu_stack.2.weight', 'linear_relu_stack.2.bias'])

In [7]:
state_dict['linear_relu_stack.0.weight']

tensor([[-0.0003,  0.0192, -0.0294,  ...,  0.0219,  0.0037,  0.0021],
        [-0.0198, -0.0150, -0.0104,  ..., -0.0203, -0.0060, -0.0299],
        [-0.0201,  0.0149, -0.0333,  ..., -0.0203,  0.0012,  0.0080],
        ...,
        [-0.0215,  0.0106,  0.0308,  ..., -0.0199,  0.0161, -0.0342],
        [ 0.0350, -0.0297, -0.0037,  ...,  0.0171,  0.0238, -0.0001],
        [ 0.0085,  0.0223, -0.0324,  ..., -0.0296,  0.0182, -0.0296]])

In [9]:
weights = []
weights.append(state_dict['linear_relu_stack.0.weight'])
weights.append(state_dict['linear_relu_stack.2.weight'])
bias = []
bias.append(state_dict['linear_relu_stack.0.bias'])
bias.append(state_dict['linear_relu_stack.2.bias'])

In [190]:
def relu(X):
    X[X<0] = 0
    return X

def d_relu(l):
    d = zs[l]
    d[zs[l] < 0] = 0
    d[zs[l] > 0] = 1
    return d

def softmax(x):
    return x.exp() / x.exp().sum()

def delta_l(y_pred, y, l):
    if l == len(weights): # This means it's the last layer... must be a vector
        y_ = torch.zeros(len(y_pred))
        y_[y] = 1
        return y_pred - y_
    else:
        return weights[l].T*d_relu(l+1)*delta_l(y_pred,y,l+1)
    
activation = []
activation.append(relu)
activation.append(softmax)
    
def forward(X):
    zs = [X]
    
    for w,b,a in zip(weights, bias, activation):
        zs.append(a(zs[-1]@w.T+b))
    return zs

def cross_entropy(y_pred,y):
    return -y_pred[y].log()

def backward(y_pred,y):
    for l in range(len(weights)):
        print(delta_l(y_pred,y,len(weights)-l).shape)
        print(zs[len(weights)-l-1].shape)
        weights[len(weights)-l-1] -= delta_l(y_pred,y,len(weights)-l)*zs[len(weights)-l-1]
        
    loss = cross_entropy(y_pred,y)



In [192]:
zs

[tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0

In [191]:
backward(y_pred,y)

torch.Size([10])
torch.Size([18])


RuntimeError: The size of tensor a (10) must match the size of tensor b (18) at non-singleton dimension 0

In [155]:
y_pred = torch.tensor([0.2,0.2,0.5,0.1,0,0,0,0,0,0])
y = 2

In [156]:
delta_l(y_pred,y,1)

torch.Size([10, 18]) torch.Size([10]) torch.Size([10])
tensor([ 0.2000,  0.2000, -0.5000,  0.1000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000])
tensor([[ 0.1390, -0.1494, -0.2147,  0.2142,  0.1166, -0.1546, -0.1654,  0.0631,
          0.1544,  0.2064,  0.0297,  0.1380,  0.0920,  0.0833, -0.1180, -0.1707,
          0.2289,  0.0489],
        [-0.2119, -0.0872,  0.0234,  0.0395, -0.1763,  0.2059, -0.2174,  0.1155,
         -0.1042, -0.1669,  0.1042, -0.0565, -0.0937, -0.0247, -0.0842,  0.2194,
         -0.2220, -0.0808],
        [-0.0286, -0.0655, -0.1446,  0.0312,  0.0048,  0.1498,  0.0774,  0.1614,
         -0.1665,  0.1966, -0.0986, -0.0030,  0.0868, -0.0487, -0.0625,  0.1071,
          0.1698, -0.1533],
        [ 0.1358,  0.1561,  0.0980,  0.0758,  0.1265,  0.2233,  0.1601,  0.0075,
         -0.1609, -0.0058, -0.1354,  0.2079, -0.0700,  0.0170,  0.0997,  0.0820,
         -0.1933, -0.1521],
        [ 0.0403, -0.1731,  0.0918,  0.0625, -0.0918, -0.1864,  0.0361, -0.131

tensor([[ 0.0278, -0.0424,  0.0143,  0.0136,  0.0000,  0.0000, -0.0000,  0.0000,
          0.0000, -0.0000],
        [-0.0299, -0.0174,  0.0328,  0.0156, -0.0000, -0.0000, -0.0000, -0.0000,
         -0.0000, -0.0000],
        [-0.0429,  0.0047,  0.0723,  0.0098,  0.0000, -0.0000, -0.0000,  0.0000,
         -0.0000,  0.0000],
        [ 0.0428,  0.0079, -0.0156,  0.0076,  0.0000, -0.0000, -0.0000, -0.0000,
          0.0000,  0.0000],
        [ 0.0233, -0.0353, -0.0024,  0.0127, -0.0000, -0.0000,  0.0000, -0.0000,
          0.0000,  0.0000],
        [-0.0309,  0.0412, -0.0749,  0.0223, -0.0000,  0.0000, -0.0000, -0.0000,
          0.0000,  0.0000],
        [-0.0331, -0.0435, -0.0387,  0.0160,  0.0000, -0.0000, -0.0000,  0.0000,
         -0.0000, -0.0000],
        [ 0.0126,  0.0231, -0.0807,  0.0008, -0.0000,  0.0000, -0.0000,  0.0000,
         -0.0000, -0.0000],
        [ 0.0309, -0.0208,  0.0832, -0.0161,  0.0000, -0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0413, -

In [110]:
zs = forward(x1)
[k.shape for k in zs]

[torch.Size([784]), torch.Size([18]), torch.Size([10])]

In [116]:
zs[-1]

tensor([0.0865, 0.0919, 0.0930, 0.0835, 0.0954, 0.0857, 0.1006, 0.1327, 0.1245,
        0.1064])